In [1]:
import numpy as np 
import pandas as pd
import time
import datetime
import gc
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from gensim.models import word2vec
import Levenshtein


In [2]:
keng_valid_26 = pd.read_csv('../result/valid_26_pred.csv')
keng_valid_29 = pd.read_csv('../result/valid_29_pred.csv')
keng_testA = pd.read_csv('../result/keng_score.csv')

yuna_valid_26 = pd.read_csv('../result/lgb1_testB_pred2.csv')
yuna_valid_29 = pd.read_csv('../result/lgb1_testB_valid.csv')
yuna_testA = pd.read_csv('../result/lgb1_testB_pred.csv')


In [22]:
valid_26_df = keng_valid_26.copy()
valid_26_df.rename(columns={'predicted_score':'keng_pred'}, inplace=True)
yuna_valid_26 = yuna_valid_26[yuna_valid_26['is_valid26'] == 1]
yuna_valid_26.reset_index(inplace=True)
valid_26_df['yuna_pred'] = yuna_valid_26['pred']

valid_29_df = keng_valid_29.copy()
valid_29_df.rename(columns={'predicted_score':'keng_pred'}, inplace=True)
valid_29_df['yuna_pred'] = yuna_valid_29['pred']

testA_df = keng_testA.copy()
testA_df.rename(columns={'predicted_score':'keng_pred'}, inplace=True)
testA_df['yuna_pred'] = yuna_testA['pred']


In [73]:
yuna_test_prefix0_df = testA_df[testA_df.is_prefix_in_train == 0].copy()
yuna_test_prefix1_df = testA_df[testA_df.is_prefix_in_train == 1].copy()
print(np.mean(yuna_test_prefix0_df['yuna_pred']))
print(np.mean(yuna_test_prefix1_df['yuna_pred']))

#定义调整函数
def resultAdjustment(result_df, t):
    result_df_temp = result_df.copy()
    result_df_temp['x'] = result_df_temp.yuna_pred.map(lambda x: -(math.log(((1 - x) / x), math.e)))
    result_df_temp['adjust_result'] = result_df_temp.x.map(lambda x: 1 / (1 + math.exp(-(x + t)))) 
    print(result_df_temp['adjust_result'].mean())
    return result_df_temp['adjust_result']

print('original mean : ', yuna_test_prefix0_df['yuna_pred'].mean())
yuna_test_df_after0 = resultAdjustment(yuna_test_prefix0_df, -0.3871)
yuna_test_df_after1 = resultAdjustment(yuna_test_prefix1_df, -0.02235)


0.44875104378627195
0.3737914647546385
original mean :  0.44875104378627195
0.37078865606571043
0.3707624986956756


In [74]:
testA_df[testA_df.is_prefix_in_train == 0]['yuna_pred'] = yuna_test_df_after0
testA_df[testA_df.is_prefix_in_train == 1]['yuna_pred'] = yuna_test_df_after1


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
valid_26 = pd.read_table('../data/oppo_round1_vali_20180926.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
valid_29 = pd.read_table('../data/oppo_round1_vali_20180929.txt', names=['prefix', 'query_prediction', 'title', 'tag', 'label'], header=None, quoting=3)
valid_26_df['label'] = valid_26['label']
valid_29_df['label'] = valid_29['label']


In [77]:
valid_26_df['predicted_score'] = valid_26_df['keng_pred'] *0.5 + valid_26_df['yuna_pred'] * 0.5
valid_29_df['predicted_score'] = valid_29_df['keng_pred'] *0.5 + valid_29_df['yuna_pred'] * 0.5


In [78]:
def getPredLabel(predArr, threshold=None, tops=None):
    '''
    根据阈值返回分类预测结果
    '''
    if tops is not None :
        temp = np.sort(np.array(predArr))
        if tops < 1:
            threshold = temp[-1*round(len(temp)*tops)]
        else:
            threshold = temp[-round(tops)]
    if threshold is None:
        print('[Error] could not get threshold value.')
        exit()
    return (predArr>=threshold).astype(int)

def findF1Threshold(predictList, labelList):
    '''
    寻找F1最佳阈值
    '''
    tempDf = pd.DataFrame({'predict':predictList, 'label':labelList})
    trueNum = len(tempDf[tempDf.label==1])
    thrList = np.unique(tempDf['predict'])
    f1List = []
    for thr in thrList:
        tempDf['temp'] = getPredLabel(tempDf['predict'], thr)
        TP = len(tempDf[(tempDf.label==1)&(tempDf.temp==1)])
        if TP==0:
            break
        positiveNum = len(tempDf[tempDf.temp==1])
        precise = TP / positiveNum
        recall = TP / trueNum
        f1 = 2 * precise * recall / (precise + recall)
        f1List.append(f1)
    f1Df = pd.DataFrame({'thr':thrList[:len(f1List)], 'f1':f1List}).sort_values(by=['f1','thr'], ascending=[False,True])
    bestThs = thrList[f1List.index(max(f1List))]
    averThr = f1Df.head(5).sort_values(by=['thr']).head(4)['thr'].mean()    # 取前5，去掉最大阈值后取平均
    # print('tops 5 thr:\n', f1Df.head(5),'aver thr:',averThr)
    return averThr




In [79]:
valid_29_prefix1_newthr = findF1Threshold(valid_29_df[valid_29_df.is_prefix_in_train == 1]['predicted_score'], valid_29_df[valid_29_df.is_prefix_in_train == 1]['label'])



In [81]:
valid_29_prefix0_newthr = findF1Threshold(valid_29_df[valid_29_df.is_prefix_in_train == 0]['predicted_score'], valid_29_df[valid_29_df.is_prefix_in_train == 0]['label'])



In [84]:
testA_df['predicted_score'] = testA_df['keng_pred'] *0.5 + testA_df['yuna_pred'] * 0.5
testA_df['label'] = 0
testA_df.loc[testA_df.is_prefix_in_train == 0, 'label'] = getPredLabel(testA_df.loc[testA_df.is_prefix_in_train == 0, 'predicted_score'], valid_29_prefix0_newthr)
testA_df.loc[testA_df.is_prefix_in_train == 1, 'label'] = getPredLabel(testA_df.loc[testA_df.is_prefix_in_train == 1, 'predicted_score'], valid_29_prefix1_newthr)


In [85]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../result/%s.csv' % fileName, header=False, index=False)

exportResult(testA_df[['label']], 'ronghe_11_6')


In [86]:
print(testA_df.head())




   is_prefix_in_train  keng_pred  yuna_pred  label  predicted_score
0                   1   0.060153   0.036359      0         0.048256
1                   1   0.702566   0.572474      1         0.637520
2                   1   0.678097   0.564478      1         0.621288
3                   1   0.286608   0.185611      0         0.236110
4                   1   0.092973   0.061782      0         0.077378


In [88]:
print(testA_df.groupby('is_prefix_in_train')[['keng_pred','yuna_pred','label','predicted_score']].describe().T)

is_prefix_in_train               0             1
keng_pred       count  4399.000000  45601.000000
                mean      0.444638      0.444645
                std       0.204084      0.325480
                min       0.022724      0.009959
                25%       0.271123      0.142847
                50%       0.414194      0.354251
                75%       0.630357      0.783005
                max       0.881357      0.993897
yuna_pred       count  4399.000000  45601.000000
                mean      0.448751      0.373791
                std       0.204494      0.313598
                min       0.039272      0.010500
                25%       0.274776      0.096333
                50%       0.437667      0.249850
                75%       0.641158      0.675860
                max       0.864953      0.982737
label           count  4399.000000  45601.000000
                mean      0.463287      0.460450
                std       0.498707      0.498439
                min 